In [24]:
'''from google.colab import drive
drive.mount('/content/drive')'''

"from google.colab import drive\ndrive.mount('/content/drive')"

In [25]:
import torch
from torchvision import datasets, models, transforms
import os
import glob
from PIL import Image
import numpy as np
import json
import torch.nn as nn
import torch.nn.functional as F

In [26]:
basePath = '/content/'
#data_dir = basePath+'Sample'
data_dir = basePath+'val2017'
drive_dir = '/content/drive/My Drive/RE/'

input_size = 224
batch_size = 2

topk = 44
filename = 'data.json'

In [27]:
#image_types = ["Covid-19","No_findings","Pneumonia"]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [28]:
'''import zipfile
import os
for file_name in os.listdir(drive_dir):
  if file_name.endswith('.zip'):
    with zipfile.ZipFile(drive_dir+file_name,'r') as zip_dir:
      zip_dir.extractall(path='/content/')'''

"import zipfile\nimport os\nfor file_name in os.listdir(drive_dir):\n  if file_name.endswith('.zip'):\n    with zipfile.ZipFile(drive_dir+file_name,'r') as zip_dir:\n      zip_dir.extractall(path='/content/')"

In [29]:
def resize_and_rename(image_dir):
    i = 1
    files = glob.glob(os.path.join(image_dir, "*"))
    for file in files:
        im = Image.open(file)
        f, e = os.path.splitext(file)
        imResize = im.resize((input_size,input_size), Image.ANTIALIAS)
        os.remove(file)

        path = '/'+os.path.join(*f.split('/')[:-1])+'/'+str(i)
        if not os.path.exists(path):
          os.mkdir(path) 
        fullPath = os.path.join(path,str(i))
        i+=1
        imResize.save(fullPath + '.png', 'PNG', quality=90)

In [30]:
'''!cp -R '/content/drive/My Drive/Sample' '/content/'
#!cp -R '/content/drive/My Drive/COVID-19/Dataset_2' '/content/'
resize_and_rename(data_dir)'''

"!cp -R '/content/drive/My Drive/Sample' '/content/'\n#!cp -R '/content/drive/My Drive/COVID-19/Dataset_2' '/content/'\nresize_and_rename(data_dir)"

In [31]:
def getTrainDataLoaders():
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    }

    print("Initializing Datasets and Dataloaders...")

    # Create training and validation datasets
    image_dataset = datasets.ImageFolder(data_dir+'/', data_transforms['val'])
    idx2label_dict = {v: k for k, v in image_dataset.class_to_idx.items()}
    # Create training and validation dataloaders
    dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    return idx2label_dict,dataloader

In [32]:
def getOutput(model):
    idx2label_dict,dataloaders = getTrainDataLoaders()
    label_lookup = np.vectorize(lambda id: np.int32(idx2label_dict[id]))

    features = np.asarray([])
    feature_labels = []

    model.eval()
    print("Features generation started...")

    i = 0
    for inputs, labels in dataloaders:
        inputs = inputs.to(device)
        #labels = labels.to(device)
        sample_fname, _ = dataloaders.dataset.samples[i]
        i += 1

        outputs = model(inputs)
        tempOut = outputs.cpu().detach().numpy()
        tempLabel = labels.cpu().detach().numpy()
        if features.shape[0] == 0:
            features = tempOut
            feature_labels = label_lookup(tempLabel)
        else:
            features = np.append(features, tempOut, axis=0)
            feature_labels = np.append(feature_labels,label_lookup(tempLabel), axis=0)
        
    return features,feature_labels

In [33]:
def ReLU(X):
   return np.maximum(0,X)

In [34]:
def CReLU(f):
    f_pos = f
    f_neg = -f
    final_f = np.concatenate((f_pos, f_neg))
    return ReLU(final_f)

In [35]:
def getSurrogateText(F,topK):
    surrText = ""
    count = topK
    for ele in F:
        instanceText = "RO"+str(ele)
        instanceText += (" "+instanceText)*(count-1)
        if len(surrText) == 0:
            surrText = instanceText
        else:
            surrText += (" "+instanceText)
        count -= 1
    return surrText

In [36]:
def F2Text(f,topK):
    rf = CReLU(f)
    perVec = np.argsort(rf)[::-1]+1
    invPer = np.argsort(perVec)+1
    after_topk = invPer[invPer <= topK]
    return getSurrogateText(after_topk,topK)

In [46]:
def getElasticData(fv,labels,topK):
    nText = fv.shape[0]
    for i in range(nText):
        #random.shuffle(li)
        surrText = F2Text(fv[i],topK)
        data1 = {"index":{"_id":i+1}}

        data2 = {
          "text": surrText,
          "image": '/'+str(labels[i])+'/'+str(labels[i])+'.png'
        }

        with open(filename,'a') as f: 
            json.dump(data1, f)
            f.write('\n')
            json.dump(data2, f)
            f.write('\n')

In [38]:
'''class VGGFeatures(nn.Module):
    def __init__(self,vgg):
        super(VGGFeatures, self).__init__()
        self.features = nn.Sequential(*list(vgg.children())[:-1])
        self.feat_vec = nn.Sequential(*list(vgg.children())[2][:-3])
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x= self.feat_vec(x)
        return x

vgg = models.vgg16(pretrained=True)
model = VGGFeatures(vgg)
model = model.to(device)'''

'class VGGFeatures(nn.Module):\n    def __init__(self,vgg):\n        super(VGGFeatures, self).__init__()\n        self.features = nn.Sequential(*list(vgg.children())[:-1])\n        self.feat_vec = nn.Sequential(*list(vgg.children())[2][:-3])\n        \n    def forward(self, x):\n        x = self.features(x)\n        x = torch.flatten(x, 1)\n        x= self.feat_vec(x)\n        return x\n\nvgg = models.vgg16(pretrained=True)\nmodel = VGGFeatures(vgg)\nmodel = model.to(device)'

In [39]:
class DenseFeatures(nn.Module):
    def __init__(self,dense):
        super(DenseFeatures, self).__init__()
        self.features = nn.Sequential(*list(dense.children())[:-1])
        #self.feat_vec = nn.Sequential(*list(dense.children())[2][:-3])
        
    def forward(self, x):
        x = self.features(x)
        x = F.relu(x, inplace=True)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        #x = torch.flatten(x, 1)
        #x= self.feat_vec(x)
        return x

dense = models.densenet121(pretrained=True)
model = DenseFeatures(dense)
model = model.to(device)

In [40]:
#print(model)

In [41]:
out,labels = getOutput(model)
print(out.shape,labels.shape)
#print(labels)

Initializing Datasets and Dataloaders...
Features generation started...
(4998, 1024) (4998,)


In [47]:
getElasticData(out,labels,topk)

In [43]:
#!cp  '/content/data.json' '/content/drive/My Drive/RE/' 